# AI Script Summary
## Why use admonition notes in your notebooks
Admonition notes (often just called "admonitions") in Jupyter Notebooks are special formatted blocks, typically implemented via Markdown extensions like MyST-Markdown or in tools like Jupyter Book, that allow you to create highlighted callouts in Markdown cells. They draw from reStructuredText (rST) styles and are used to emphasize certain content without disrupting the flow of the notebook.

### Primary Reasons for Using Admonitions
- **Highlighting Key Information**: They make it easy to spotlight tips, notes, or other advisory content in a visually distinct way, improving readability for educational materials, tutorials, or documentation. For example, a "note" admonition might provide additional context or a quick fact.
- **Warnings and Cautions**: Admonitions are ideal for alerting users to potential pitfalls, errors, or important caveats, such as deprecated code or safety reminders in data analysis workflows. This helps prevent mistakes in interactive or shared notebooks.
- **Structuring Complex Content**: In longer notebooks, they organize information like citations, figures, or side explanations, making the document more professional and easier to navigate. Tools like Jupyter Book extend this for book-like outputs.
- **Customization and Rendering**: They support custom fences (e.g., :::) for better compatibility across interfaces, reducing clutter in raw Markdown while enabling rich rendering in HTML or PDF exports.

The "Note" below is an example of the style for GBR Modelling script repo admonition note. Templated styles can be generated using the `generate_admonition_template` function in this notebook.

## Libraries and data

In [ ]:
# Libraries
# ==============================================================================
from openai import OpenAI
# import openai

from IPython.display import Markdown, display

import pyperclip

import os

import requests

import nbformat

import json

## Processing 

These scripts collectively enable the automated reading, parsing, and AI-driven summarization of Jupyter notebooks by extracting content from code, markdown, and output cells, then using a conversational interface powered by an OpenAI-compatible API to generate initial summaries and handle follow-up queries while maintaining context.

### Descriptions of Each Function / Class

- **`read_notebook(notebook_path)`**: This function opens and parses a Jupyter notebook file (.ipynb) from the specified path using nbformat, returning a structured NotebookNode object that represents the notebook's contents in version 4 format.

- **`extract_content(notebook)`**: This function iterates through all cells in a given notebook, extracting and formatting the source code from code cells (including any text outputs or results), as well as markdown content from markdown cells, and combines them into a single concatenated string separated by descriptive headers like "# Code cell:" or "# Markdown:".
---
- The **`NotebookSummarizer`** class is a Python wrapper that facilitates conversational summarization of Jupyter notebooks using the OpenAI API via OpenRouter, maintaining a persistent message history for context-aware initial summaries and follow-up queries while limiting history length to manage token usage.

- **`__init__(self, api_key, model="openai/gpt-4o", max_history=10)`** (method of NotebookSummarizer): This constructor initializes the NotebookSummarizer class by creating an OpenAI client configured for OpenRouter, setting the model and history limit, and starting the conversation history with a system prompt that instructs the AI to act as an expert in summarizing Jupyter notebooks while maintaining context.

- **`initial_summarize(self, prompt)`** (method of NotebookSummarizer): This method begins the summarization process by optionally copying the provided prompt (notebook content) to the system clipboard, appending it as a user message to the conversation history, and then calling the internal _get_response method to obtain and return the AI's initial summary.

- **`follow_up(self, query)`** (method of NotebookSummarizer): This method allows for subsequent interactions by appending a new user query to the history, trimming the history if it exceeds the maximum length by removing the oldest user-assistant pairs (while preserving the system prompt), and then retrieving and returning the AI's response based on the updated context.

- **`_get_response(self)`** (method of NotebookSummarizer): This private method handles the core API interaction by sending the current conversation history to the OpenAI chat completions endpoint via the configured client, extracting the assistant's response text (with fallback handling for different response formats), appending it to the history, and returning it, while including optional headers and a high max_tokens limit for detailed outputs; it raises an exception if the extraction fails.

In [15]:
def read_notebook(notebook_path):
    """
    Reads a Jupyter notebook file and returns its content.
    
    Args:
        notebook_path (str): Path to the .ipynb file
        
    Returns:
        nbformat.NotebookNode: Parsed notebook object
    """
    with open(notebook_path, 'r', encoding='utf-8') as file:
        return nbformat.read(file, as_version=4)
    

def extract_content(notebook):
    """
    Extracts text from code and markdown cells in a notebook.
    
    Args:
        notebook (nbformat.NotebookNode): The notebook to process
        
    Returns:
        str: Combined content from all cells
    """
    content_parts = []
    
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            # Include both the code and any text outputs
            content_parts.append(f"# Code cell:\n{cell.source}\n")
            if 'outputs' in cell and cell.outputs:
                for output in cell.outputs:
                    if output.output_type == 'stream' and 'text' in output:
                        content_parts.append(f"# Output:\n{output.text}\n")
                    elif output.output_type == 'execute_result' and 'data' in output:
                        if 'text/plain' in output.data:
                            content_parts.append(f"# Result:\n{output.data['text/plain']}\n")
        elif cell.cell_type == 'markdown':
            content_parts.append(f"# Markdown:\n{cell.source}\n")
    
    return "\n".join(content_parts)


class NotebookSummarizer:
    """
    A conversational wrapper for summarizing notebooks with persistent context.
    Manages message history for follow-up queries.
    """
    def __init__(self, api_key, model="openai/gpt-4o", max_history=10):
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )
        self.model = model
        self.history = []  # List of {"role": str, "content": str}
        self.max_history = max_history  # Limit to prevent token overflow
        # Initial system message
        system_prompt = 'You are an expert at analyzing and summarizing Jupyter notebooks for data science and programming contexts. Maintain context from previous interactions.'
        self.history.append({"role": "system", "content": system_prompt})
    
    def initial_summarize(self, prompt):
        """
        Start the conversation with the initial notebook content.
        
        Args:
            content (str): The notebook content to summarize.
        """
        # Build initial prompt (your original, but without repeating in follow-ups)

        pyperclip.copy(prompt)  # Optional: Copy for manual use
        self.history.append({"role": "user", "content": prompt})
        return self._get_response()
    
    def follow_up(self, query):
        """
        Send a follow-up query using the existing context.
        
        Args:
            query (str): The follow-up question or instruction (e.g., "Explain the ARD method in more detail").
        
        Returns:
            str: The assistant's response.
        """
        self.history.append({"role": "user", "content": query})
        # Trim history if too long (remove oldest user/assistant pairs)
        while len(self.history) > self.max_history + 1:  # +1 for system
            self.history = [self.history[0]] + self.history[2:]  # Skip first user/assistant pair
        return self._get_response()
    
    def _get_response(self):
        """
        Internal method to call the API and extract response.
        Uses fallback extraction for robustness.
        """
        def extract_text_from_response(response_obj):
            # Same as previous: Extract from ChatCompletion or raw JSON
            if hasattr(response_obj, 'choices') and response_obj.choices:
                return response_obj.choices[0].message.content
            # Fallback recursion (omitted for brevity; use the previous extract_text_from_response)
            return ""
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.history,
                extra_headers={
                    "HTTP-Referer": "your-app-url",  # Optional
                    "X-Title": "Notebook Summarizer",
                },
                max_tokens=9000,
            )
            assistant_response = extract_text_from_response(response)
            if not assistant_response:
                raise ValueError("No text extracted from response")
            # Append to history
            self.history.append({"role": "assistant", "content": assistant_response})
            return assistant_response
        except Exception as e:
            # Fallback to alpha endpoint if needed (adapt as in previous code)
            print(f"API call failed: {e}")
            raise

## Prompts
The `prompt_prefix` is a structured template designed for AI analysis of Jupyter notebooks, directing the generation of a detailed summary organized under six specific headings. 

In contrast, the `definition_text` provides concise instructions for creating a brief 2-3 line notebook summary to insert into a predefined HTML snippet.

In [11]:
prompt_prefix = f"""
        Please analyze the following Jupyter notebook content and provide a comprehensive summary.
        Organise your response under the following headings:
        1. The main purpose and objectives of the notebook
        2. Key code logic and functions implemented
        3. Important findings or results shown in outputs
        4. Overall structure and flow
        5. Instructions for use
        6. Theoretical Description of Methods

        Consistently format the heading like "## 1. The main purpose and objectives of the notebook\n\n" 

        Provide a bibliography of web references if used.

        Write mathematical expressions using proper LaTeX syntax. Format as inline ($) or display ($$) equations. Do not escape backslashes.
        
        Be careful to avoid KaTeX parse errors like 'Expected EOF'.

        Avoid overuse of bullet or numbered lists.

        At the end, add an interesting and possibly relevant fact with a reference if available.
        
        """


definition_text = """
    write a short two to three line summary of the notebook and paste it into the following html snippet
    relpacing the field '[short_summary]'

> ## <strong style="color:#00b8d4; font-size:28px;">AI Script Summary</strong>
> <span style="color:#757575; font-size:18px; display:block; margin-top:1px;">[short_summary] </span> <br/><br/>
> <strong>Authors:</strong> F. R. Bennett &nbsp;&nbsp; <br/><br/>
> <strong>Date:</strong> 17/10/25  &nbsp;&nbsp; <br/><br/>
> <strong>Version:</strong> 1.0<br/><br/>
> 
> <button onclick="handleGitHubAction('frbennett', 'shapleyx', 'Examples/ishigami_new_legendre.ipynb', 'download')">Download File</button>
> 
><button onclick="handleGitHubAction('frbennett', 'shapleyx', 'Examples', 'download')">Download Folder</button>
>
><button onclick="handleGitHubAction('frbennett', 'shapleyx', 'Examples/ishigami_new_legendre.ipynb', 'open')">Open on GitHub</button>
> <br/><br/>

<script src="https://cdnjs.cloudflare.com/ajax/libs/jszip/3.10.0/jszip.min.js"></script>
<script>
async function handleGitHubAction(owner, repo, path, action) {
  const apiUrl = `https://api.github.com/repos/${owner}/${repo}/contents/${path}`;
  const githubUrl = `https://github.com/${owner}/${repo}/tree/main/${path}`;

  if (action === 'open') {
    window.open(githubUrl, '_blank');
    return;
  }

  const response = await fetch(apiUrl);
  const data = await response.json();

  if (Array.isArray(data)) {
    // Directory download
    const zip = new JSZip();
    for (const file of data) {
      if (file.type === "file") {
        const fileRes = await fetch(file.download_url);
        const content = await fileRes.text();
        zip.file(file.name, content);
      }
    }
    const blob = await zip.generateAsync({ type: "blob" });
    const link = document.createElement("a");
    link.href = URL.createObjectURL(blob);
    link.download = `${path.split('/').pop()}.zip`;
    link.click();
  } else if (data.type === "file") {
    // Single file download
    const decoded = atob(data.content.replace(/\n/g, ''));
    const blob = new Blob([decoded], { type: 'application/octet-stream' });
    const link = document.createElement('a');
    link.href = URL.createObjectURL(blob);
    link.download = data.name;
    link.click();
  } else {
    alert("Unsupported content type or path not found.");
  }
}
</script>


                    """ 

<div class="admonition example" name="html-admonition" style="background: rgba(92,107,192,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #5c6bc0; border-color: #5c6bc0; padding-left: 10px; padding-right: 10px;">
<p class="title">
    <i style="font-size: 18px; color:#5c6bc0;">&#128221;</i>
    <b style="color: #5c6bc0;">Example</b>
</p>
<p>
The <code>summarise_this</code> function serves as a demonstration of the previously defined <code>NotebookSummarizer</code> class and related utilities by taking a path to a Jupyter notebook file and an optional AI model (defaulting to "google/gemini-2.5-flash-lite"), reading and extracting its content, constructing a prompt with a predefined prefix, and generating both an initial summary and a follow-up definition response using the summarizer. 
</p>
</div>

In [19]:
def summarise_this(NOTEBOOK_PATH, model="google/gemini-2.5-flash-lite"):
    # Configuration
    OPENAI_API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Get API key from environment

    if not OPENAI_API_KEY:
        raise ValueError("Please set the OPENAI_API_KEY environment variable")

    if not os.path.exists(NOTEBOOK_PATH):
        raise FileNotFoundError(f"Notebook file not found: {NOTEBOOK_PATH}")

    # Read and process the notebook
    print("Reading notebook...")
    notebook = read_notebook(NOTEBOOK_PATH)

    print("Extracting content...")
    notebook_content = extract_content(notebook)

    # Build prompt

    prompt = f"""
    {prompt_prefix}
            
    Notebook content:
    {notebook_content}

    """


    summarizer = NotebookSummarizer(api_key=OPENAI_API_KEY, model=model)
    summary = summarizer.initial_summarize(prompt)

    
    definition_response = summarizer.follow_up(definition_text)

    # Usage

    return summary, definition_response

<div class="admonition example" name="html-admonition" style="background: rgba(92,107,192,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #5c6bc0; border-color: #5c6bc0; padding-left: 10px; padding-right: 10px;">
<p class="title">
    <i style="font-size: 18px; color:#5c6bc0;">&#128221;</i>
    <b style="color: #5c6bc0;">Example</b>
</p>
<p>
Now, pulling it all together and building a summary of this notebook. The final composed result is displayed using the <code>Markdown</code> function and is saved to the clipboard using <code>pyperclip</code> to be pasted directly into a markdown document or cell. 
</p>
</div>

In [25]:
model="x-ai/grok-4-fast"

summary, definition_response = summarise_this(current_path, model=model)

all_result = f""" 
{definition_response}
# Detailed Summary
---
{summary}
"""
pyperclip.copy(all_result)
Markdown(all_result)


Reading notebook...


PermissionError: [Errno 13] Permission denied: 'd:\\gdrive\\My Drive\\Work Projects\\Coding_Projects\\script_repo\\scripts\\utilities'

In [ ]:
NOTEBOOK_PATH = 'ai_script_summary.ipynb'
# Read and process the notebook
print("Reading notebook...")
notebook = read_notebook(NOTEBOOK_PATH)

print("Extracting content...")
notebook_content = extract_content(notebook)

# Build prompt

prompt = f"""
{prompt_prefix}
        
Notebook content:
{notebook_content}

"""
pyperclip.copy(prompt)

Reading notebook...
Extracting content...


In [ ]:
pyperclip.copy(summary)

In [ ]:
pyperclip.copy(all_result)

In [ ]:
with open("output.md", "w") as file:
    file.write(all_result)

---


> ## <strong style="color:#00b8d4; font-size:28px;">AI Script Summary</strong>
> <span style="color:#757575; font-size:18px; display:block; margin-top:1px;">This notebook implements an automated Jupyter notebook summarization tool that extracts content from code and markdown cells, then processes it through various large language models via the OpenRouter API. It provides structured summaries with consistent formatting and includes utilities for result export and prompt debugging. The system supports multiple AI models and handles the complete pipeline from notebook parsing to formatted output generation. </span>
>
> <strong>Authors:</strong> F. R. Bennett &nbsp;&nbsp; <br/><br/>
> <strong>Date:</strong> 17/10/25  &nbsp;&nbsp; <br/><br/>
> <strong>Version:</strong> 1.0


> ## <strong style="color:#00b8d4; font-size:28px;">AI Script Summary</strong>
> <span style="color:#757575; font-size:18px; display:block; margin-top:1px;">This Jupyter notebook conducts global sensitivity analysis on the Ishigami function, a nonlinear benchmark for uncertainty quantification, using RS-HDMR from the shapleyx library. It generates Sobol-sampled inputs, fits a sparse second-order polynomial surrogate via ARD with cross-validation, and derives sensitivity indices like Sobol, SHAP, PAWN, HX, and deltaX to assess variable contributions and interactions. Key findings reveal X2's strong main effect (Sobol 0.446) and X1-X3 interaction (0.240), with near-perfect model reconstruction (R² ≈ 1.000). </span>
>
> <strong>Authors:</strong> F. R. Bennett &nbsp;&nbsp; <br/><br/>
> <strong>Date:</strong> 17/10/25  &nbsp;&nbsp; <br/><br/>
> <strong>Version:</strong> 1.0<br/><br/>
> <a href="https://github.com/frbennett/shapleyx/blob/main/Examples/ishigami_new_legendre.ipynb">View notebook in Github</a>
> <br/><br/>
> <a href="https://minhaskamal.github.io/DownGit/#/home?url=https://github.com/frbennett/shapleyx/blob/main/Examples/ishigami_new_legendre.ipynb">Download notebook</a>
> <br/><br/>
                    

> ## <span style="font-size: 25px; line-height: 1.0; color: #ff7043; margin-right: 100px;">&#9888;<span> <strong style="color: #ff7043;">Caution</strong>
> Purpose: sssss
> <br/><br/>


<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #9888;">&#9998 Note</b>
</p>

See <a href="../faq/cyclical-features-time-series.html" target="_blank">Cyclical features in time series forecasting</a> for a more detailed description of strategies for encoding cyclic features.

</div>

In [ ]:
def generate_admonition_template(admonition_type: str, content: str = None) -> str:
    """
    Generate an HTML admonition note template based on the specified type.
    
    Args:
        admonition_type (str): The type of admonition (e.g., "Note", "Tip").
        content (str, optional): The content to insert inside the admonition. Defaults to a placeholder.
    
    Returns:
        str: The generated HTML string for the admonition.
    
    Raises:
        ValueError: If the admonition_type is not in ADMONITION_TEMPLATES.
    """
    ADMONITION_TEMPLATES = {
        "Note": {"icon": "&#9998;", "color": "#00b8d4", "label_color": "#00b8d4"},
        "Tip": {"icon": "&#127775;", "color": "#4caf50", "label_color": "#4caf50"},
        "Info": {"icon": "&#8505;", "color": "#1976d2", "label_color": "#1976d2"},
        "Success": {"icon": "&#10004;", "color": "#2e7d32", "label_color": "#2e7d32"},
        "Warning": {"icon": "&#9888;", "color": "#fbc02d", "label_color": "#fbc02d"},
        "Danger": {"icon": "&#10060;", "color": "#d32f2f", "label_color": "#d32f2f"},
        "Caution": {"icon": "&#9888;", "color": "#ff7043", "label_color": "#ff7043"},
        "Question": {"icon": "&#10067;", "color": "#6a1b9a", "label_color": "#6a1b9a"},
        "Hint": {"icon": "&#128161;", "color": "#00897b", "label_color": "#00897b"},
        "Example": {"icon": "&#128221;", "color": "#5c6bc0", "label_color": "#5c6bc0"},
        "Important": {"icon": "&#128295;", "color": "#455a64", "label_color": "#455a64"},
        "Deprecated": {"icon": "&#128221;", "color": "#ff5722", "label_color": "#ff5722"},
        "Experimental": {"icon": "&#9881;", "color": "#9c27b0", "label_color": "#9c27b0"},
        "Performance": {"icon": "&#128200;", "color": "#3f51b5", "label_color": "#3f51b5"},
        "Reference": {"icon": "&#128214;", "color": "#607d8b", "label_color": "#607d8b"},
    }
    
    if admonition_type not in ADMONITION_TEMPLATES:
        raise ValueError(f"Admonition type '{admonition_type}' not supported. Available types: {list(ADMONITION_TEMPLATES.keys())}")
    
    template = ADMONITION_TEMPLATES[admonition_type]
    icon = template["icon"]
    color = template["color"]
    label_color = template["label_color"]
    
    # Convert hex color to RGB for background
    def hex_to_rgb(hex_color: str) -> tuple:
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    
    r, g, b = hex_to_rgb(color)
    bg_style = f"background: rgba({r},{g},{b},.1);"
    
    # Default content if none provided
    if content is None:
        content = f"Replace this with your {admonition_type.lower()} content."
    
    # Build the HTML
    html = f'''<div class="admonition {admonition_type.lower()}" name="html-admonition" style="{bg_style} padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid {color}; border-color: {color}; padding-left: 10px; padding-right: 10px;">
<p class="title">
    <i style="font-size: 18px; color:{color};">{icon}</i>
    <b style="color: {label_color};">{admonition_type}</b>
</p>
<p>{content}</p>
</div>'''
    
    return html

# Example usage:
# print(generate_admonition_template("Note", "See <a href='../faq/example.html' target='_blank'>Example</a> for details."))
# Output: The full HTML for a Note admonition with the provided content.


In [ ]:
test = generate_admonition_template('Note', 'Thi is a string')

In [ ]:
pyperclip.copy(test)

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px; padding-right: 10px;">
<p class="title">
    <i style="font-size: 18px; color:#00b8d4;">&#9998;</i>
    <b style="color: #00b8d4;">Note</b>
</p>
<p>Thi is a string</p>
</div>